This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get(f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-01&api_key={API_KEY}')

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2017-01-01',
  'end_date': '2017-01-01',
  'frequency': 'daily',
  'data': [],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#code task 1 Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
r = requests.get(f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}')

In [6]:
#code task 2 Convert the returned JSON object into a Python dictionary.
afxx2017 = r.json()
type(afxx2017)

dict

In [7]:
#short data analysis on data
print(len(afxx2017['dataset_data']['data'])) #number of data points available
print(afxx2017['dataset_data']['data'][0]) #cross checking singular data point with api return
print(type(afxx2017['dataset_data']['data'][0][5]))
afxx = afxx2017['dataset_data']['data'] #naming variable to dataset for less clutter

255
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]
<class 'NoneType'>


In [8]:
'''
code task 3
Calculate what the highest and lowest opening prices were for the stock in this period.

Assigned first values as both highest and lowest.
if not empty value
check if its the highest and update the global variable. same for lowest
'''

highest, lowest = afxx[0][1], afxx[0][1]
for i in range(len(afxx)):
    x = afxx[i][1]
    if x == None:
        continue
    if x > highest:
        highest = x
    if x < lowest:
        lowest = x

(highest, lowest)

(53.11, 34.0)

In [9]:
'''
code task 4
What was the largest change in any one day (based on High and Low price)?

high = high value of current day
low = low value of current day
check if either value is missing
find the largest change by doing high - low
compare change for that day with current largest delta and update the value if necessary
'''

max_delta = 0
for i in range(len(afxx)):
    high = afxx[i][2]
    low = afxx[i][3]
    if high == None:
        continue
    if low == None:
        continue
    delta = high - low
    if delta > max_delta:
        max_delta = delta
        
max_delta

2.8100000000000023

In [10]:
'''
code task 5
What was the largest change between any two days (based on Closing Price)?

d1 = closing price first day
d2 = closing price second day
confirm neither value is missing
take absolute value of difference to ensure it is taking the greatest change whether positive or negative
compare change for that day with current largest delta and update the value if necessary
'''
max_delta = 0
for i in range(len(afxx)-1):
    d1 = afxx[i][4]
    d2 = afxx[i+1][4]
    if d1 == None:
        continue
    if d2 == None:
        continue
    delta = abs(d2-d1)
    if delta > max_delta:
        max_delta = delta

max_delta

2.559999999999995

In [11]:
'''
code task 6
What was the average daily trading volume during this year?

total_trading = total sum of all trading volume this year
trading_count = total number of non-None data points
tv = trading volume of the day

confirm if tv is None
update total_trading by adding tv, update count variable as well
'''

total_trading = 0
trading_count = 0
for i in range(len(afxx)):
    tv = afxx[i][5]
    if tv == None:
        continue
    total_trading += tv
    trading_count += 1
avg = total_trading/trading_count

avg

-0.44